## Random Forest with StandardScaler and optuna on dataset2.0
dataset: 11 features, normalizsation of useful vote removed

20220926

In [1]:
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import optuna as opt
import warnings
warnings.filterwarnings("ignore")
import time

In [2]:
#import data
train_df = pd.read_parquet("train_main.parquet.snappy")
test_df = pd.read_parquet("test_main.parquet.snappy")
val_df = pd.read_parquet("val_main.parquet.snappy")

In [3]:
X_train = train_df.drop(['r_id','r_useful'],axis=1)
y_train = train_df['r_useful']
X_test = test_df.drop(['r_id','r_useful'],axis=1)
y_test = test_df['r_useful']
X_val = val_df.drop(['r_id','r_useful'],axis=1)
y_val = val_df['r_useful']

# standardise the features
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [6]:
def objective(trial):
    start = time.time()
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 1, 500, 50),
        "max_depth": trial.suggest_int('max_depth', 1, 500, 50),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 20),
        "max_features" : trial.suggest_categorical("max_features", ['sqrt', 'log2']),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20), 
        "random_state": 760, 
        "n_jobs": -1
    }
    print("Currently running with:")
    print(params)
    
    model = RandomForestRegressor(**params)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print("Time cost:", time.time()-start)
    return mean_squared_error(y_val, y_pred, squared=False)

In [7]:
study = opt.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
study.best_params

[I 2022-09-26 17:07:54,407] A new study created in memory with name: no-name-792aad30-3e29-4e5a-a262-fd6fb6e389f9


Currently running with:
{'n_estimators': 1, 'max_depth': 151, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 8, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-26 17:07:59,122] Trial 0 finished with value: 3.9679400799878497 and parameters: {'n_estimators': 1, 'max_depth': 151, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 8}. Best is trial 0 with value: 3.9679400799878497.


Time cost: 4.710000276565552
Currently running with:
{'n_estimators': 401, 'max_depth': 1, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-26 17:08:33,939] Trial 1 finished with value: 3.961959658873423 and parameters: {'n_estimators': 401, 'max_depth': 1, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1}. Best is trial 1 with value: 3.961959658873423.


...

Currently running with:
{'n_estimators': 401, 'max_depth': 401, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 3, 'random_state': 760, 'n_jobs': -1}
Time cost: 305.42399883270264


[I 2022-09-26 19:53:04,055] Trial 48 finished with value: 3.3021344527146477 and parameters: {'n_estimators': 401, 'max_depth': 401, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 3}. Best is trial 30 with value: 3.2849702774778318.


Currently running with:
{'n_estimators': 301, 'max_depth': 351, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 1, 'random_state': 760, 'n_jobs': -1}
Time cost: 231.83333611488342


[I 2022-09-26 19:56:56,337] Trial 49 finished with value: 3.288576658416398 and parameters: {'n_estimators': 301, 'max_depth': 351, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 1}. Best is trial 30 with value: 3.2849702774778318.


{'n_estimators': 251,
 'max_depth': 401,
 'min_samples_split': 5,
 'max_features': 'log2',
 'min_samples_leaf': 1}

In [8]:
#optimal model
# n_estimators': 251, 'max_depth': 401, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 1
omodel = RandomForestRegressor(n_estimators = 251, 
                               max_depth = 401, 
                               min_samples_leaf= 1,
                               min_samples_split= 5,
                               max_features= 'log2', random_state = 760, n_jobs=-1)
omodel.fit(X_train, y_train)
#predict train set results
y_pred = omodel.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
mae = mean_absolute_error(y_train, y_pred) 
print(f"\n model train score -> RMSE:", rmse, "MAE:", mae)

#predict validation set results
y_pred_val = omodel.predict(X_val)
rmse = mean_squared_error(y_val, y_pred_val, squared=False)
mae = mean_absolute_error(y_val, y_pred_val) 
print(f"\n model val score -> RMSE:", rmse, "MAE:", mae)

#predict test set results
y_pred_test = omodel.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_test, squared=False)
mae = mean_absolute_error(y_test, y_pred_test) 
print(f"\n model test score -> RMSE:", rmse, "MAE:", mae)


 model train score -> RMSE: 2.1576167544353106 MAE: 0.7695524561956715

 model val score -> RMSE: 3.2849702774778318 MAE: 1.5578832094537869

 model test score -> RMSE: 3.60868735954618 MAE: 1.570931916082599
